In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [2]:
url_base='https://www.cgmh.org.tw/tw/Systems/'
url_work_table='RecruitInfo/3'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
pages=soup.find_all('ul' ,class_="layout__pagination ul-reset")[0]
def get_pages(pages):
  counter=0
  for i, item in enumerate(pages):
      if item.find('a') :
        s = item.find('a')
        counter += 1
        try:
          #print(s.get("href") ,s.string)
          if s.get("href") == "javascript:void(0)":
            counter -= 1
        except:
          counter -= 1
          #print(item,'error')
      
  return counter
get_pages(pages)


3

In [4]:
def get_work_dead_line( soup):  
  work_detail_web = soup.find('article').get_text().replace("\xa0","").replace('\n',"").replace("\u3000","")
  if '額滿為止' in work_detail_web:
    return '額滿為止'
  elif '即日起收件至' in work_detail_web:
    dead_line = work_detail_web.rsplit("即日起收件至")[1].split("止")[0].replace("：","")
    return dead_line      
  else:
    dead_line = work_detail_web.rsplit("報名期限")[1].split("截止")[0].replace("：","")
    return dead_line

In [6]:
def get_work_page(page):
    g=requests.get(url+'?page='+str(page))
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [7]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup    

In [8]:
def get_work_table(soup,tables,work_table):
    for i, item in enumerate(tables):
        if item.find('a'): #過濾掉被刪除的文章
            s = item.find('a')
            url_base_website = urlparse(url)
            work_detail_link = url_base_website._replace(path=urlparse(s.get('href')).path).geturl()
            title = item.find_all('div')[1].string
            work_page_soup = get_work_detail(work_detail_link)
            dead_line = get_work_dead_line( work_page_soup )
            print('#{}召聘職稱: {} 期限: {}\n 連結：{}'.format(i+1, title, dead_line, work_detail_link ))
            work_table.append([i-2, title, dead_line, work_detail_link ])

In [11]:
work_table=[]
for _page in range(1,get_pages(pages)+1):
    soup_=get_work_page(_page)
    tables = soup_.find('div',class_="bg-grey pd100").find_all('ul')[-2].find_all('li')
    get_work_table(soup_,tables,work_table)

#1召聘職稱: 新北市立土城醫院呼吸治療科招募呼吸治療師二名，請 查照。 期限: 2022年04月20日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070023
#2召聘職稱: 林口長庚醫院檢驗醫學部招募抽血技術員，請 查照。 期限: 2022年04月17日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070012
#3召聘職稱: 新北市立土城醫院麻醉科招募麻醉技術員，請 查照。 期限: 2022年04月25日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060024
#4召聘職稱: 嘉義長庚紀念醫院招募定期契約戒菸衛教師人員，請 查照。 期限: 2022年4月20日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204080010
#5召聘職稱: 新北市立土城醫院檢驗醫學科招募定期契約醫事檢驗師，請查照。 期限: 2022年05月07日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060059
#6召聘職稱: 嘉義長庚醫院招募血液透析治療員人員，請 查照。 期限: 111年04月30日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060055
#7召聘職稱: 基隆長庚醫院牙科招募牙科護理技術員3名，請查照。 期限: 2022年04月25日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204010037
#8召聘職稱: 基隆長庚紀念醫院招募臨床心理師，請 查照。 期限: 2022年4月14日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202203300006
#9召聘職稱: 林口長庚醫院招募自費麻醉學員 期限: 2022年04月17日
 連結：https://

In [ ]:

work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

    no                           召聘職稱           期限  \
0   -2   新北市立土城醫院呼吸治療科招募呼吸治療師二名，請 查照。  2022年04月20日   
1   -1       林口長庚醫院檢驗醫學部招募抽血技術員，請 查照。  2022年04月17日   
2    0       新北市立土城醫院麻醉科招募麻醉技術員，請 查照。  2022年04月25日   
3    1    嘉義長庚紀念醫院招募定期契約戒菸衛教師人員，請 查照。   2022年4月20日   
4    2  新北市立土城醫院檢驗醫學科招募定期契約醫事檢驗師，請查照。  2022年05月07日   
..  ..                            ...          ...   
95   3        嘉義長庚醫院招募血液透析治療員人員，請 查照。   111年04月30日   
96   4       基隆長庚醫院牙科招募牙科護理技術員3名，請查照。  2022年04月25日   
97   5          基隆長庚紀念醫院招募臨床心理師，請 查照。   2022年4月14日   
98   6                 林口長庚醫院招募自費麻醉學員  2022年04月17日   
99   7    新北市立土城醫院影像診療科招募放射診斷照護員，請查照。  2022年04月18日   

                                                   連結  
0   https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
1   https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
2   https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
3   https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
4   https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
..             

In [ ]:
for f in range(1,get_pages(pages)+1):
    print(f)

1
2
3
